In [1]:
import numpy as np
import pandas as pd

In [2]:
E = 2.1e11
v = 0.29
dn = 1.17411e-10
di = 1.17563e-10
psi_i = 0.7854

In [3]:
sigma_phi = (E/((1+v)*(np.sin(psi_i)**2))) * ((di-dn)/dn )

In [4]:
sigma_phi * 1e-6

421.4953480743183

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
from ipywidgets import interact, widgets
from pathlib import Path
import os

p = Path()
p = p/'src'/'difr'

In [6]:
filenames = [file for file in os.listdir(p)]
frames = dict()
for filename in filenames:
        filepath = p/filename
        try:
            frames[filepath.name] = pd.read_csv(
                filepath,
                skiprows=33,
                delim_whitespace = True,
                names=['2Theta','I'],
                engine='python'
            )
        except FileNotFoundError:
            pass

In [7]:
interact(lambda key: frames[key].plot(x = '2Theta', y = 'I'), key = list(frames.keys()))

interactive(children=(Dropdown(description='key', options=('PDF04_Grupo1440_2Pts_.txt', 'PDF04_Grupo1440_2Pts_…

<function __main__.<lambda>(key)>

In [8]:
def fit_curve(key, degree):
    df = frames[key]
    fit = np.polyfit(df['2Theta'], df['I'], degree)
    return fit

In [9]:
def calc_curve(key, deg):
    x = np.linspace(152,160,80)
    y = 0
    for i, parameter in enumerate(fit_curve(key, deg)):
        y += parameter * x**(deg-i)
    return x, y

In [10]:
def plot_curve(key, deg):
    df = frames[key]
    fig = go.Figure()
    fig.add_traces(go.Scatter(x = df['2Theta'], y = df['I'], name = 'Data', mode = 'markers+lines'))
    fig.add_traces(go.Scatter(x = calc_curve(key, deg)[0], y = calc_curve(key, deg)[1], name = 'Fit', mode = 'lines'))
    max_index = np.argmax(calc_curve(key, deg)[1])
    angle_max = calc_curve(key, deg)[0][max_index]
    fig.add_vline(
        x = angle_max,
        annotation_text='Máx = {:.2f} deg'.format(angle_max),
        annotation_position="top right"
    )
    return fig 

In [11]:
interact(plot_curve, key = list(frames.keys()), deg = int(6))

interactive(children=(Dropdown(description='key', options=('PDF04_Grupo1440_2Pts_.txt', 'PDF04_Grupo1440_2Pts_…

<function __main__.plot_curve(key, deg)>